In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from dateutil.parser import parse
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA

In [2]:
transactions_full = pd.read_csv('./data/transactions_train.csv', parse_dates=["TX_TS"])
# transactions_train_og = transactions_train_og.set_index("TX_ID")

merchants_full = pd.read_csv('./data/merchants.csv')

terminals_full = pd.read_csv('./data/terminals.csv')

customers_full = pd.read_csv('./data/customers.csv')

C:\Users\joshu\AppData\Local\Temp\ipykernel_9404\1249260220.py:1: DtypeWarning: Columns (18) have mixed types. Specify dtype option on import or set low_memory=False.
  transactions_full = pd.read_csv('./data/transactions_train.csv', parse_dates=["TX_TS"])


In [3]:
transactions_full = pd.merge(transactions_full,merchants_full,on='MERCHANT_ID')

In [4]:
transactions_full = pd.merge(transactions_full,terminals_full,on='TERMINAL_ID')

In [5]:
transactions_full = pd.merge(transactions_full,customers_full,on='CUSTOMER_ID')

In [6]:
useless_columns = ['MERCHANT_ID','TERMINAL_ID','CUSTOMER_ID','CARD_EXPIRY_DATE','TX_ID','CARD_DATA','LEGAL_NAME','FOUNDATION_DATE','ACTIVE_FROM','TRADING_FROM']
transactions_full.drop(useless_columns,axis=1,inplace=True)

In [7]:
transactions_full.dropna(subset=['ACQUIRER_ID','CARDHOLDER_AUTH_METHOD'], inplace=True)

In [8]:
transactions_full['ANNUAL_TURNOVER_CASH'] = transactions_full['ANNUAL_TURNOVER'] - transactions_full['ANNUAL_TURNOVER_CARD']

In [9]:
transactions_full.iloc[1].TX_TS

Timestamp('2021-01-27 06:43:38+0000', tz='UTC')

In [10]:
bins = [0, 6, 12, 18, 24]
labels = ['00:00-05:59', '06:00-11:59', '12:00-17:59', '18:00-23:59']
transactions_full['Time_of_day'] = pd.cut(transactions_full.TX_TS.dt.hour, bins, labels=labels, right=False)

In [11]:
bins = np.arange(12)
transactions_full['Month'] = transactions_full['TX_TS'].apply(lambda x: x.month)

In [12]:
transactions_full['Day_of_week'] = transactions_full['TX_TS'].apply(lambda x: x.weekday())

In [13]:
transactions_full.drop(['TX_TS'],axis=1,inplace=True)

In [14]:
transactions_full.loc[transactions_full['IS_RECURRING_TRANSACTION'] == 'True',['IS_RECURRING_TRANSACTION']] = 1
transactions_full.loc[transactions_full['IS_RECURRING_TRANSACTION'] == True,['IS_RECURRING_TRANSACTION']] = 1
transactions_full.loc[transactions_full['IS_RECURRING_TRANSACTION'] == 'False',['IS_RECURRING_TRANSACTION']] = 0
transactions_full.loc[transactions_full['IS_RECURRING_TRANSACTION'] == False,['IS_RECURRING_TRANSACTION']] = 0

transactions_full['IS_RECURRING_TRANSACTION'] = transactions_full['IS_RECURRING_TRANSACTION'].apply(pd.to_numeric)

In [15]:
one_hot_encoded_col = ['CARD_BRAND','TRANSACTION_TYPE','TRANSACTION_STATUS','FAILURE_CODE','FAILURE_REASON','TRANSACTION_CURRENCY','CARD_COUNTRY_CODE','ACQUIRER_ID','CARDHOLDER_AUTH_METHOD','BUSINESS_TYPE','OUTLET_TYPE','Time_of_day']
transactions_full = pd.get_dummies(transactions_full, columns = one_hot_encoded_col)

In [16]:
location = 'feature_set_2'
transactions_full.to_csv(F'./data/{location}/transactions_full.csv',index=False)

In [17]:
# Divide data into training and validation subsets
y = transactions_full['TX_FRAUD']
X = transactions_full.drop(['TX_FRAUD'],axis = 1)
X_train_full, X_valid_full, y_train, y_valid = train_test_split(X, y, train_size=0.7, test_size=0.3,random_state=42)

In [18]:
# X_train_full.to_csv(F'./data/{location}/X_train_full.csv',index=False)
# X_valid_full.to_csv(F'./data/{location}/X_valid_full.csv',index=False)
# y_train.to_csv(F'./data/{location}/y_train.csv',index=False)
# y_valid.to_csv(F'./data/{location}/y_valid.csv',index=False)

In [19]:
scaler = MinMaxScaler()
scaler.fit(transactions_full)

MinMaxScaler()

In [20]:
transactions_full_normalised = pd.DataFrame(scaler.transform(transactions_full), index=transactions_full.index, columns=transactions_full.columns)

In [33]:
# Divide data into training and validation subsets
location = 'feature_set_2_normalised'
y = transactions_full_normalised['TX_FRAUD']
X = transactions_full_normalised.drop(['TX_FRAUD'],axis = 1)
X_train_full, X_valid_full, y_train, y_valid = train_test_split(X, y, train_size=0.7, test_size=0.3,random_state=42)


transactions_full_normalised.to_csv(F'./data/{location}/transactions_full.csv',index=False)
X_train_full.to_csv(F'./data/{location}/X_train_full.csv',index=False)
X_valid_full.to_csv(F'./data/{location}/X_valid_full.csv',index=False)
y_train.to_csv(F'./data/{location}/y_train.csv',index=False)
y_valid.to_csv(F'./data/{location}/y_valid.csv',index=False)

In [35]:
for num in np.arange(20,80,5):
  pca = PCA(n_components=num)
  pca.fit(transactions_full_normalised.drop(['TX_FRAUD'],axis = 1))
  print(F'{num} {sum(pca.explained_variance_ratio_)}')

20 0.7130571502425729
25 0.7875557698590858
30 0.8476553283152235
35 0.8908937151760492
40 0.9230460279485373
45 0.951224152736609
50 0.9712399490648517
55 0.9824923883626412
60 0.9894925128359442
65 0.9942337394426755
70 0.9971491717209402
75 0.9990519675996908


In [36]:
pca = PCA(n_components=65)
pca.fit(X)

PCA(n_components=65)

In [37]:
# number of components
n_pcs= pca.components_.shape[0]

# get the index of the most important feature on EACH component
# LIST COMPREHENSION HERE
most_important = [np.abs(pca.components_[i]).argmax() for i in range(n_pcs)]

initial_feature_names = pca.feature_names_in_
# get the names
most_important_names = [initial_feature_names[most_important[i]] for i in range(n_pcs)]

# LIST COMPREHENSION HERE AGAIN
dic = {'PC{}'.format(i): most_important_names[i] for i in range(n_pcs)}

# build the dataframe
df = pd.DataFrame(dic.items())

In [38]:
subset_pca_data = transactions_full_normalised[most_important_names]

In [39]:
# Divide data into training and validation subsets
X_train_full, X_valid_full, y_train, y_valid = train_test_split(subset_pca_data, y, train_size=0.7, test_size=0.3,random_state=42)

In [41]:
X_train_full.to_csv('./data/feature_set_2_normalized_pca/X_train_full.csv',index=False)
X_valid_full.to_csv('./data/feature_set_2_normalized_pca/X_valid_full.csv',index=False)
y_train.to_csv('./data/feature_set_2_normalized_pca/y_train.csv',index=False)
y_valid.to_csv('./data/feature_set_2_normalized_pca/y_valid.csv',index=False)